Ideas:

1. Physics simulator with mechanics (blocks) from images and rotating bodies (&their images)
2. GridWorld go to goal, with few obstacles (so that the resulting fcn is simple)

In [88]:
from copy import deepcopy
import numpy as np
import math

In [123]:
class KeyChestEnvironment(object):
    OBJECTS = ['empty', 'wall', 'player', 'key', 'chest', 'food', 'lamp_on', 'lamp_off']
    SYMBOLS = {'wall': '#', 'player': 'P', 'key': '<', 'chest': '>', 'food': '@',
               'lamp_on': 'L', 'lamp_off': 'l', 'empty': ' '}
    
    def __init__(self, labyrinth_maps, initial_health, food_efficiency,
                 food_rows=None, keys_rows=None):
        """Environment with keys and chests."""
        self.initial_maps = deepcopy(labyrinth_maps)
        self.maps = deepcopy(labyrinth_maps)
        self.shape = self.maps[self.SYMBOLS['empty']].shape
        assert set(self.maps.keys()) == set(KeyChestEnvironment.SYMBOLS.values())
        self.player_position = self.locate_single(self.SYMBOLS['player'])
        self.keys = 0
        self.food_rows = food_rows
        self.keys_rows = keys_rows
        self.initial_health = initial_health
        self.health = initial_health
        self.width = self.shape[1]
        self.height = self.shape[0]
        self.first_render = True
        
        # to see if everything fits
        self.render()
        
    def render(self):
        sx, sy = self.shape
        
        if self.first_render:
            maxfood = self.initial_health + np.sum(self.initial_maps[self.SYMBOLS['food']])
            maxkeys = np.sum(self.initial_maps[self.SYMBOLS['key']])
            food_rows = round(math.ceil(1. * maxfood / (2 + self.width)))
            keys_rows = round(math.ceil(1. * maxkeys / (2 + self.width)))

            if self.food_rows is None:
                self.food_rows = food_rows
            else:
                assert self.food_rows >= food_rows

            if self.keys_rows is None:
                self.keys_rows = keys_rows
            else:
                assert self.keys_rows >= keys_rows
                
            print(self.keys_rows, self.food_rows)
            self.first_render = False
        
        dy1 = 1
        dy2 = 1
        dy = dy1 + dy2
        dx1 = 1 + self.food_rows + self.keys_rows
        dx2 = 1
        dx = dx1 + dx2
                
        shape = (sx + dx, sy + dy)
        out = np.full(fill_value=self.SYMBOLS['empty'], shape=shape, dtype='<U1')
        
        out[dx1 - 1, :] = self.SYMBOLS['wall']
        out[-1, :] = self.SYMBOLS['wall']
        out[dx1 - 1:, 0] = self.SYMBOLS['wall']
        out[dx1 - 1:, -1] = self.SYMBOLS['wall']
        for obj in self.OBJECTS:
            symb = self.SYMBOLS[obj]
            mask = self.maps[symb] > 0
            out[dx1:-dx2,dy1:-dy2][mask] = self.SYMBOLS[obj]
        return out
        
    def locate(self, object_type):
        return KeyChestEnvironment._locate(self.maps, object_type)
    
    def locate_single(self, object_type):
        return KeyChestEnvironment._locate_single(self.maps, object_type)
    
    @staticmethod
    def _locate(maps, object_type):
        """Where are objects of a given type on the map?"""
        assert object_type in KeyChestEnvironment.SYMBOLS.values(), \
            f"Wrong object {object_type}, have {maps.keys()}"
        return list(zip(*np.where(maps[object_type])))
    
    @staticmethod
    def _locate_single(maps, object_type):
        """Where is the single object?"""
        w = KeyChestEnvironment._locate(maps, object_type)
        assert len(w) >= 1, f"No {object_type} found"
        assert len(w) <= 1, f"More than one {object_type} found"
        return w[0]

In [124]:
class KeyChestEnvironmentRandom(KeyChestEnvironment):
    
    def __init__(self, width=10, height=10, n_keys=2, n_chests=2, n_food=2, **kwargs):
        objects_to_fill = ['player']
        objects_to_fill += ['key'] * n_keys
        objects_to_fill += ['chest'] * n_chests
        objects_to_fill += ['food'] * n_food
        shape = (height, width)
        wh = width * height
        assert wh >= len(objects_to_fill)
        
        positions = []
        for i in range(height):
            for j in range(width):
                positions.append((i, j))
                
        pos_select = np.random.choice(range(len(positions)), len(objects_to_fill), replace=False)
        
        maps = {k: np.zeros(shape, dtype=np.bool) for k in self.SYMBOLS.values()}
                
        for pos, obj in zip(pos_select, objects_to_fill):
            m = maps[self.SYMBOLS[obj]]
            p = positions[pos]
            m[p[0], p[1]] = True
        
        super(KeyChestEnvironmentRandom, self).__init__(labyrinth_maps=maps, **kwargs)

In [125]:
env = KeyChestEnvironmentRandom(initial_health=10, food_efficiency=5)

1 1


In [128]:
for r in env.render():
    print(''.join(r))

            
            
############
#  <   <   #
#       >  #
#         P#
#          #
#          #
#          #
#          #
#       >@ #
#   @      #
#          #
############


In [118]:
env.food_rows, env.keys_rows

(1, 1)